In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
import re
import unicodedata
import fitz  # PyMuPDF
from tqdm import tqdm
from langchain.schema import Document as LangchainDocument
from PyPDF2 import PdfReader
from typing import List
from typing import Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv

c:\year3term2\gardio_chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
# ดึง API Token
api_key = os.getenv("HUGGINGFACE_API_KEY")

In [4]:
import os
import pytesseract

# กำหนดพาธไปยัง tesseract.exe (ปรับให้ตรงกับตำแหน่งที่ติดตั้งในเครื่องของคุณ)
tesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
if os.path.exists(tesseract_path):
    pytesseract.pytesseract.tesseract_cmd = tesseract_path
else:
    print(f"Tesseract ไม่พบที่ {tesseract_path}. กรุณาติดตั้ง Tesseract OCR และตรวจสอบ PATH.")


In [5]:
import os
import re
from tqdm import tqdm
import PyPDF2
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
from langchain.docstore.document import Document  # ใช้ Document ของ LangChain

# กำหนดพาธไปยัง tesseract.exe (สำหรับ Windows)
if os.name == 'nt':  # ตรวจสอบว่าเป็น Windows
    tesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    if os.path.exists(tesseract_path):
        pytesseract.pytesseract.tesseract_cmd = tesseract_path
    else:
        print(f"Tesseract ไม่พบที่ {tesseract_path}. กรุณาติดตั้ง Tesseract OCR และตรวจสอบ PATH.")

# ฟังก์ชันดึงข้อความและ metadata จาก PDF ด้วย PyPDF2
def extract_text_from_pdf_with_metadata(pdf_path):
    text = ""
    metadata = {}
    try:
        pdf_path = os.path.normpath(pdf_path)
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            metadata = reader.metadata or {}
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text, metadata

# ฟังก์ชันทำความสะอาดข้อความ (ลบเว้นวรรคส่วนเกิน)
def clean_text(text):
    return " ".join(text.split())

# ฟังก์ชัน post-process สำหรับข้อความภาษาไทย
def postprocess_thai_text(text):
    # ลบเว้นวรรคระหว่างตัวอักษรไทย (Unicode \u0E00-\u0E7F)
    return re.sub(r'(?<=[\u0E00-\u0E7F])\s+(?=[\u0E00-\u0E7F])', '', text)

# ฟังก์ชันใช้ OCR ดึงข้อความจาก PDF ด้วย PyMuPDF (ไม่ต้องใช้ poppler)
def ocr_pdf_with_pymupdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            # สร้าง pixmap ด้วย DPI สูงขึ้นเพื่อความคมชัด (300 dpi)
            pix = page.get_pixmap(dpi=300)
            mode = "RGB" if pix.alpha == 0 else "RGBA"
            # สร้าง PIL Image จาก pixmap
            img = Image.frombytes(mode, (pix.width, pix.height), pix.samples)
            # ใช้ pytesseract OCR พร้อมระบุภาษาไทยและ config --psm 6
            page_text = pytesseract.image_to_string(img, lang='tha')
            # ทำ post-process เพื่อลบเว้นวรรคเกินในภาษาไทย
            # page_text = postprocess_thai_text(page_text)
            text += page_text + "\n"
        doc.close()
    except Exception as e:
        print(f"Error during OCR with PyMuPDF on {pdf_path}: {e}")
    return text

# รายการไฟล์ PDF ที่ต้องการประมวลผล (ใช้ raw string เพื่อป้องกันปัญหา escape sequence)
pdf_files = [
    r"C:\year3term2\gardio_chatbot\backend\vector_store\documents\การพัฒนาระบบเพื่อประเมินความผิดปกติทางด้านพฤติกรรมหมาแมว.pdf",
    r"C:\year3term2\gardio_chatbot\backend\vector_store\documents\พัฒนาการของพฤติกรรมการเรียนรู้ของสัตว์_สุนัข_แมว.pdf",
    r"C:\year3term2\gardio_chatbot\backend\vector_store\documents\รวมคำถามที่พบบ่อยเกี่ยวกับพฤติกรรมหมาแมว.pdf",
]

RAW_KNOWLEDGE_BASE = []

for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    # พยายาม extract ข้อความด้วย PyPDF2 ก่อน
    text, metadata = extract_text_from_pdf_with_metadata(pdf_file)
    # print(f"\nExtracted text from {pdf_file}:\n{'-'*40}\n{text}\n{'-'*40}")
    
    # หากไม่ได้ข้อความ (หรือได้ค่าว่าง) ให้ใช้ OCR ด้วย PyMuPDF
    if not text.strip():
        # print(f"No text extracted from {pdf_file}, trying OCR with PyMuPDF...")
        text = ocr_pdf_with_pymupdf(pdf_file)
        # print(f"OCR result from {pdf_file}:\n{'-'*40}\n{text}\n{'-'*40}")
    
    # ทำความสะอาดข้อความ
    cleaned_text = clean_text(text)
    # print(f"Cleaned text from {pdf_file}:\n{'-'*40}\n{cleaned_text}\n{'-'*40}")
    
    # สร้าง Document ด้วย LangChain แล้วเก็บลง RAW_KNOWLEDGE_BASE
    doc = Document(page_content=cleaned_text, metadata={"source": pdf_file, "details": metadata})
    RAW_KNOWLEDGE_BASE.append(doc)

print("\nRAW_KNOWLEDGE_BASE:")
for doc in RAW_KNOWLEDGE_BASE:
    print(doc)


Processing PDFs: 100%|██████████| 3/3 [17:29<00:00, 349.75s/it]


RAW_KNOWLEDGE_BASE:
page_content='5 อ 6 อ 6 ผู ่ 56 น ร ร 0 ท ร , ร ใ ล 1 ไร, ล ท ส ล น ไท อ ว ห อ ก ๊ แอ ร ก อะ ป ู ท 1 ร เน ๒ พ์ ๕ ล ฟ์ เอ ท ล ะ : ท เป 0 ร :// แ ห พ พนม. ห ล ร ส ล ห 6 ท ธ ู ล ไล . ท ล!/ เ น ๒ พ์ 6 ล น อ ท /380295971 เพ -“ า า ณ ณ ณ์ แ โ โต ร ณ์ ณ์ ณ์ แ ณี แพ พ ณ แ ไห น พ้อ ๕ ไอ «| ไล ง 2024 6 ๐ ห ณะ แอ พ ร ค ะ เวร 0 16 2 ล น ป ท อ แร, ! ท 6 ๕ ไน ต ไท ธ : โน ล 6 ทา อ ท ธ เฟ ฟ อ ว ท ธ ู เ ล พ ล ท พ ฟุ ฟ ล ไ ล ! ไ ล แ : ป ท เ โง อ ร 1 ไห 113 ท บ ธน 6 ๕110 พ ร 486 6 เก 1 แอ พ ร 5 ธ 6 ธ 6 6 ด 0 ห ศ แ น แซ 1 ๕ ๐ ก แอ ท 1! โอ || อ พ เ ไท ธ ู ฝ่า 15 ต ล 6 เพ ล ร น ฏ ไ 0 ส ฝ ่ อด ู ่ 0 โน ๕ ก า อ ท ธ เฟ อ ก ธ เ (ส พ พ ล ท อ ท 03 ! ไ ล ง 2024. โท ด น ร ล ท ล ร ห อ ส ุ น ล ร ไธ ฝ ่ อ ท ท ล ท 6 ๕ ท า ล ท 1 ๐ ์ ์ ป ท 6 ต ่ อ พ เท!|0 ล ฝ ่ อ ยู่ ก ๊ ไอ . ไจ 6 ร 6 ล 6 ไล เ 6 = < ) | ” ร “ว ิ ท ย า ศา ส ต ร ์ เท ค โน โล ย ี แล ะ น ว ั ต ก ร ร ม ส ู ่ ก า ร พ ั ฒ น า ท ี ย ั ง ย ื น ” 5 ม ี น า ค ม 2567 ณ โร ง แร ม ร อ ย ั ล ร ิ เว อ ร ์ ก ร ุ ง เท พ ฯ ก า ร พ ั ฒ น า ร ะ บ บ เพ ื 

In [6]:
print(f"ประเภทของ RAW_KNOWLEDGE_BASE: {type(RAW_KNOWLEDGE_BASE)}")
print(f"จำนวนเอกสารใน RAW_KNOWLEDGE_BASE: {len(RAW_KNOWLEDGE_BASE)}")
if isinstance(RAW_KNOWLEDGE_BASE, list) and RAW_KNOWLEDGE_BASE:
    print(f"ประเภทของเอกสารตัวแรก: {type(RAW_KNOWLEDGE_BASE[0])}")
    print(f"เนื้อหาใน page_content ตัวแรก: {RAW_KNOWLEDGE_BASE[0].page_content[:500]}")


ประเภทของ RAW_KNOWLEDGE_BASE: <class 'list'>
จำนวนเอกสารใน RAW_KNOWLEDGE_BASE: 3
ประเภทของเอกสารตัวแรก: <class 'langchain_core.documents.base.Document'>
เนื้อหาใน page_content ตัวแรก: 5 อ 6 อ 6 ผู ่ 56 น ร ร 0 ท ร , ร ใ ล 1 ไร, ล ท ส ล น ไท อ ว ห อ ก ๊ แอ ร ก อะ ป ู ท 1 ร เน ๒ พ์ ๕ ล ฟ์ เอ ท ล ะ : ท เป 0 ร :// แ ห พ พนม. ห ล ร ส ล ห 6 ท ธ ู ล ไล . ท ล!/ เ น ๒ พ์ 6 ล น อ ท /380295971 เพ -“ า า ณ ณ ณ์ แ โ โต ร ณ์ ณ์ ณ์ แ ณี แพ พ ณ แ ไห น พ้อ ๕ ไอ «| ไล ง 2024 6 ๐ ห ณะ แอ พ ร ค ะ เวร 0 16 2 ล น ป ท อ แร, ! ท 6 ๕ ไน ต ไท ธ : โน ล 6 ทา อ ท ธ เฟ ฟ อ ว ท ธ ู เ ล พ ล ท พ ฟุ ฟ ล ไ ล ! ไ ล แ : ป ท เ โง อ ร 1 ไห 113 ท บ ธน 6 ๕110 พ ร 486 6 เก 1 แอ พ ร 5 ธ 6 ธ 6 6 ด 0 ห ศ แ น แซ 1 ๕ ๐ ก 


In [7]:
from langchain.schema import Document

if not isinstance(RAW_KNOWLEDGE_BASE, list):
    RAW_KNOWLEDGE_BASE = [RAW_KNOWLEDGE_BASE]  # แปลงเป็นลิสต์
elif isinstance(RAW_KNOWLEDGE_BASE, list) and len(RAW_KNOWLEDGE_BASE) > 0:
    RAW_KNOWLEDGE_BASE = [
        doc if isinstance(doc, Document) else Document(page_content=str(doc))
        for doc in RAW_KNOWLEDGE_BASE
    ]


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3", use_fast=True)


In [9]:
from typing import List, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
# กำหนดโมเดลสำหรับ Tokenization
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(
    chunk_size: int,
    knowledge_base: List,  # ใช้ List แทน LangchainDocument 
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List:
    """แยกเอกสารเป็นชิ้นขนาดสูงสุด `chunk_size` tokens"""
    
    # โหลด tokenizer
    try:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
    except Exception as e:
        print(f"❌ Error loading tokenizer: {e}")
        return []

    # ตั้งค่า Text Splitter
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    # แยกเอกสารเป็น Chunk
    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # ลบเอกสารที่ซ้ำกัน
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

docs_processed = split_documents(
    512,  # ขนาด chunk
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

if docs_processed:
    print(f"✅ แบ่งเอกสารได้ {len(docs_processed)} ชิ้น")
else:
    print("❌ ไม่พบข้อมูล หรือมีปัญหาในการแบ่งเอกสาร")


✅ แบ่งเอกสารได้ 2500 ชิ้น


In [10]:
from huggingface_hub import login
access_token = api_key
login(access_token)

EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# ตรวจสอบว่า docs_processed ไม่ว่าง
if not docs_processed:
    raise ValueError("docs_processed ไม่มีข้อมูล กรุณาตรวจสอบเอกสารก่อนสร้าง FAISS index")

# สร้าง FAISS index
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

# บันทึก FAISS ลงไฟล์
KNOWLEDGE_VECTOR_DATABASE.save_local("faiss_index")